In [1]:
!nvidia-smi

Mon Jul 26 20:56:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.81       Driver Version: 466.81       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P3    12W /  N/A |    105MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x14a8a007f40>)

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
x = torch.ones(2, 3, device=try_gpu())
x

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
y = torch.rand(2, 3, device=try_gpu(1))
y

tensor([[0.3637, 0.6247, 0.1023],
        [0.7099, 0.1625, 0.5194]])

In [8]:
z = y.cuda(0)
print(x)
print(z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[0.3637, 0.6247, 0.1023],
        [0.7099, 0.1625, 0.5194]], device='cuda:0')


In [9]:
x + z

tensor([[1.3637, 1.6247, 1.1023],
        [1.7099, 1.1625, 1.5194]], device='cuda:0')

In [10]:
z.cuda(0) is z

True

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [12]:
net(x)

tensor([[0.9562],
        [0.9562]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)